## problema caxeiro viajante aluguel de carros 

Backtracking e Branch-and-Bound

# Instance creator

In [5]:
import random 

def generate_symmetric_matrix(size):
    matrix = [[0 for _ in range(size)] for _ in range(size)]
    for i in range(size):
        for j in range(i, size):
            if(i!=j):
                value = random.randint(1, 100)  # Random integer between 1 and 100
                matrix[i][j] = value
                matrix[j][i] = value  # Make it symmetric
    return matrix

def generate_cars_instance_file(citys, cars, file_name):
    with open(file_name, 'w') as file:
        file.write(f"NAME : Test{citys}n\n")
        file.write("TYPE : CaRS\n")
        file.write("COMMENT : Instances for the CaRS Problem (Asconavieta & Goldbarg)\n")
        file.write(f"DIMENSION : {citys}\n")
        file.write(f"CARS_NUMBER : {cars}\n")
        file.write("EDGE_WEIGHT_TYPE : EXPLICIT\n")
        file.write("EDGE_WEIGHT_FORMAT : FULL_MATRIX\n")

        file.write("EDGE_WEIGHT_SECTION\n")
        for i in range(cars):
            file.write(f"{i}\n")
            matrix = generate_symmetric_matrix(citys)
            for row in matrix:
                for n in row:
                    file.write(f"{n} ")
                file.write("\n")


        file.write("RETURN_RATE_SECTION\n")
        for i in range(cars):
            file.write(f"{i}\n")
            matrix = generate_symmetric_matrix(citys)
            for row in matrix:
                for n in row:
                    file.write(f"{n} ")
                file.write("\n")

        file.write("EOF\n")

# Example usage:
generate_cars_instance_file(6,2, "teste.car")


# Instance reader

In [6]:
class CaRSInstance:
    def __init__(self):
        self.name = ""
        self.type = ""
        self.comment = ""
        self.dimension = 0
        self.cars_number = 0
        self.edge_weight_type = ""
        self.edge_weight_format = ""

class CaRSCar:
    def __init__(self, car_id):
        self.car_id = car_id
        self.distance_matrix = []
        self.return_rate_matrix = []

def read_cars_instance(file_path):
    cars_instance = CaRSInstance()
    cars = []  # List to store Car objects

    with open(file_path, 'r') as file:
        lines = file.readlines()

    section = None
    car_id = None  # To keep track of the current car
    edge_weight_section = False
    return_rate_section = False

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if line.startswith("EDGE_WEIGHT_SECTION"):
            edge_weight_section = True
            section = "EDGE_WEIGHT"
            continue
        elif line.startswith("RETURN_RATE_SECTION"):
            return_rate_section = True
            edge_weight_section = False
            section = "RETURN_RATE"
            continue
        elif line.startswith("EOF"):
            section = None  # Mark the end of the section
            continue

        if edge_weight_section:
            if line.isdigit():
                car_id = int(line)
                cars.append(CaRSCar(car_id))
            else:
                values = line.split()
                cars[-1].distance_matrix.append([int(value) for value in values])
        elif return_rate_section:
            if line.isdigit():
                car_id = int(line)
            else:
                values = line.split()
                cars[car_id].return_rate_matrix.append([int(value) for value in values])

    for line in lines:
        key_value = line.split(":")
        if len(key_value) == 2:
            key, value = key_value
            key = key.strip()
            value = value.strip()

            if key == "NAME":
                cars_instance.name = value
            elif key == "TYPE":
                cars_instance.type = value
            elif key == "COMMENT":
                cars_instance.comment = value
            elif key == "DIMENSION":
                cars_instance.dimension = int(value)
            elif key == "CARS_NUMBER":
                cars_instance.cars_number = int(value)
            elif key == "EDGE_WEIGHT_TYPE":
                cars_instance.edge_weight_type = value
            elif key == "EDGE_WEIGHT_FORMAT":
                cars_instance.edge_weight_format = value

    return cars_instance, cars

# Usage example
file_path = "teste.car"  # Replace with the path to your input file
# file_path = "/Users/tarsila/Documents/paa/PCA-main 2/CaRS_NaoEuclidianas/simple.car"  # Replace with the path to your input file
# file_path = "/Users/tarsila/Documents/paa/PCA-main 2/CaRS_NaoEuclidianas/Bolivia10n.car"  # Replace with the path to your input file
cars_instance, cars = read_cars_instance(file_path)

# Access the parsed data
print("Name:", cars_instance.name)
print("Type:", cars_instance.type)
print("Comment:", cars_instance.comment)
print("Dimension:", cars_instance.dimension)
print("Cars Number:", cars_instance.cars_number)
print("Edge Weight Type:", cars_instance.edge_weight_type)
print("Edge Weight Format:", cars_instance.edge_weight_format)


Name: Test8n
Type: CaRS
Comment: Instances for the CaRS Problem (Asconavieta & Goldbarg)
Dimension: 8
Cars Number: 2
Edge Weight Type: EXPLICIT
Edge Weight Format: FULL_MATRIX


In [7]:

# Access the distance matrices for each car
for car in cars:
    print(f"Car {car.car_id} Distance Matrix:")
    for row in car.distance_matrix:
        print(row)

# Access the return rate matrices for each car
for car in cars:
    print(f"Car {car.car_id} Return Rate Matrix:")
    for row in car.return_rate_matrix:
        print(row)

Car 0 Distance Matrix:
[0, 26, 99, 81, 10, 11, 83, 72]
[26, 0, 99, 48, 39, 37, 44, 55]
[99, 99, 0, 94, 18, 18, 72, 1]
[81, 48, 94, 0, 73, 95, 48, 8]
[10, 39, 18, 73, 0, 100, 72, 80]
[11, 37, 18, 95, 100, 0, 75, 55]
[83, 44, 72, 48, 72, 75, 0, 61]
[72, 55, 1, 8, 80, 55, 61, 0]
Car 1 Distance Matrix:
[0, 95, 1, 34, 50, 47, 7, 31]
[95, 0, 81, 19, 47, 12, 6, 23]
[1, 81, 0, 74, 82, 79, 45, 73]
[34, 19, 74, 0, 7, 6, 16, 48]
[50, 47, 82, 7, 0, 96, 80, 52]
[47, 12, 79, 6, 96, 0, 18, 41]
[7, 6, 45, 16, 80, 18, 0, 47]
[31, 23, 73, 48, 52, 41, 47, 0]
Car 0 Return Rate Matrix:
[0, 60, 25, 99, 100, 36, 73, 54]
[60, 0, 97, 18, 36, 84, 51, 41]
[25, 97, 0, 39, 27, 99, 90, 38]
[99, 18, 39, 0, 13, 23, 21, 8]
[100, 36, 27, 13, 0, 42, 30, 97]
[36, 84, 99, 23, 42, 0, 67, 72]
[73, 51, 90, 21, 30, 67, 0, 76]
[54, 41, 38, 8, 97, 72, 76, 0]
Car 1 Return Rate Matrix:
[0, 99, 100, 75, 7, 30, 54, 60]
[99, 0, 87, 76, 91, 80, 64, 99]
[100, 87, 0, 62, 9, 71, 67, 51]
[75, 76, 62, 0, 60, 58, 4, 3]
[7, 91, 9, 60, 0, 93

## Backtracking

In [8]:
import itertools
import time

def calculate_total_distance(route, car_assignments, cars):
    total_distance = 0
    current_car = car_assignments[0]
    current_location = 0  # Comece em qualquer cidade (0 é geralmente a primeira cidade)

    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]

        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]

        current_location = next_location

    return total_distance

def backtrack_tsp_with_cars(instance, cars):
    num_cars = instance.cars_number
    num_locations = instance.dimension

    best_route = None
    best_distance = float('inf')

    for route in itertools.permutations(range(1, num_locations), num_locations - 1):
        for car_assignments in itertools.product(range(num_cars), repeat=num_locations):
            # Certifique-se de que a rota começa e termina na cidade 0
            full_route = (0,) + route + (0,)

            distance = calculate_total_distance(full_route, car_assignments, cars)

            if distance < best_distance:
                best_distance = distance
                best_route = full_route
                best_car_assignments = car_assignments

    return best_route, best_distance, best_car_assignments

start_time = time.time()

best_route, best_distance, best_car_assignments = backtrack_tsp_with_cars(cars_instance, cars)

end_time = time.time()
print("Tempo de execução:", end_time - start_time, "segundos")

print("Melhor rota:", best_route)
print("Melhor distância:", best_distance)
print("best_car_assignments:", best_car_assignments)



Tempo de execução: 3.8580222129821777 segundos
Melhor rota: (0, 2, 6, 1, 5, 3, 4, 7, 0)
Melhor distância: 160
best_car_assignments: (1, 1, 1, 1, 1, 1, 1, 1)


## Branch-and-Bound

In [9]:
from itertools import permutations
from enum import Enum

class TreeNode:
    def __init__(self, value, remaining_elements=None):
        self.value = value
        self.children = []
        self.remaining_elements = remaining_elements
        self.type = 1
        self.father_value = None


    def delete(self):
        self.remove(self)

    def delete_child(self, child_node):
        if child_node in self.children:
            self.children.remove(child_node)

    def delete_by_target(self, target):
        for child in self.children:
            if child == target:
                self.children.remove(child)
                return True
            if child.delete_by_target(target):
                return True

    def __iter__(self):
        yield self
        for child in self.children:
            yield from child

    def find_by_value(self, target_value):
        if self.value == target_value:
            return self

        for child in self.children:
            result = child.find_by_value(target_value)
            if result:
                return result
            
    def find_sequences_by_value(self, target_value, position):
        results = [] 
        if verificar_sequencia(self.value, target_value, position):
            results.append(self)

        for child in self.children:
            child_results = child.find_sequences_by_value(target_value, position)
            results.extend(child_results)

        return results

    def find_sequences_by_values(self, target_value, father, position):
        if self.value == target_value:
            return self
        all_results = set()

        for child in self.children:
            results = child.find_sequences_by_value(father, position)
            if results:
                for result in results:
                    result2 = result.find_sequences_by_value(target_value, position)
                    if result2:
                        all_results.add(result2[0])
        return 
    
    def find_place_sequences_by_value(self, target_value, position):
        results = [] 
        if verificar_sequencia(self.value, target_value, position):
            results.append(self)
        if(self.father_value == None):
            for child in self.children:
                child_results = child.find_sequences_by_value(target_value, position)
                results.extend(child_results)
        return results
    
    def delete_sequences_by_value(self, target_value, position, father):
        if verificar_sequencia(self.value, target_value, position):
            father.children.remove(self)
        for child in self.children:
            child.delete_sequences_by_value(target_value, position, self)

    
    def delete_sequences_by_values(self, target_value, father, position):
        for child in self.children:
            results_fathers = child.find_place_sequences_by_value(father, position)
            if results_fathers:
                for result in results_fathers:
                    result.delete_sequences_by_value(target_value, position, self)

    
    def count_leaves(self):
        if not self.children:
            return 1
        else:
            leaf_count = 0
            for child in self.children:
                leaf_count += child.count_leaves()
            return leaf_count
    

def verificar_sequencia(sequence, seq, i):
    if sequence[i:i+len(seq)] == seq:
        return True
    else:
        return False

def build_permutation_tree_2(node, num_locations, num_cars, father_value):
    if len(node.value) == num_locations:
        node.type = 2
        node.father_value = father_value
        return
    for element in range(num_cars):
        new_value =  node.value + (element, )
        child_node = TreeNode(new_value)
        node.children.append(child_node)
        node.type = 2
        node.father_value = father_value
        build_permutation_tree_2(child_node, num_locations, num_cars,father_value)

def build_permutation_tree(node, num_locations, num_cars):
    if len(node.value) == num_locations - 1:
        child_node = TreeNode((), tuple(range(1, num_locations)))
        node.children.append(child_node)
        build_permutation_tree_2(child_node, num_locations, num_cars, node.value)
        return
    for element in node.remaining_elements:
        new_value = node.value + (element,)
        new_remaining_elements = [e for e in node.remaining_elements if e != element]
        child_node = TreeNode(new_value, new_remaining_elements)
        node.children.append(child_node)
        build_permutation_tree(child_node, num_locations, num_cars)

def print_tree(node, prefix=""):
    print(prefix + str(node.value))
    for child in node.children:
        print_tree(child, prefix + "  ")


def delete_nodes_with_sequence( root, target_value, father , size):
    for s in range(size):
        root.delete_sequences_by_values( target_value, father, s)
    return root



In [10]:
import itertools

def calculate_total_distance(route, car_assignments, cars, best_distance):
    total_distance = 0
    current_car = car_assignments[0]
    current_location = 0 
    stop_position = 0
    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]
        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
        if best_distance < total_distance:
            return total_distance, stop_position

        current_location = next_location
        stop_position+= 1

    return total_distance, stop_position


def branch_bound_tsp_with_cars(instance, cars):

    num_cars = instance.cars_number
    num_locations = instance.dimension
    root = TreeNode((), tuple(range(1, num_locations)))

    build_permutation_tree(root, num_locations, num_cars)
    best_route = None
    best_distance = float('inf')
    best_car_assignments = []

    for node in root:
        if(node.father_value != None and len(node.value) == num_locations):
            full_route = (0,) + node.father_value + (0,)
            car_assignments = node.value
            distance, stop_position = calculate_total_distance(full_route, car_assignments, cars, best_distance)
            if distance < best_distance:
                best_distance = distance
                best_route = full_route
                best_car_assignments = car_assignments
            elif stop_position < len(car_assignments) - 1: 
                car_sequence = car_assignments[:stop_position]
                place_sequence =  node.father_value[:stop_position]

                root = delete_nodes_with_sequence(root,car_sequence,place_sequence, num_locations - stop_position)

    return best_route, best_distance, best_car_assignments

start_time = time.time()

best_route, best_distance, best_car_assignments = branch_bound_tsp_with_cars(cars_instance, cars)
end_time = time.time()
print("Tempo de execução:", end_time - start_time, "segundos")
print("Melhor rota:", best_route)
print("Melhor distância:", best_distance)
print("best_car_assignments:", best_car_assignments)